# High-level TF Example

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from common.params import *
from common.utils import *

In [2]:
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = "1"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.3
Tensorflow:  1.4.0
GPU:  ['Tesla K80']


In [4]:
def create_symbol(training):
    """ TF pooling requires a boolean flag for dropout, faster when using
    'channels_first' for data_format """
    conv1 = tf.layers.conv2d(X, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu1 = tf.nn.relu(conv1)
    conv2 = tf.layers.conv2d(relu1, filters=50, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    pool1 = tf.layers.max_pooling2d(conv2, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    relu2 = tf.nn.relu(pool1)
    drop1 = tf.layers.dropout(relu2, 0.25, training=training)
    
    conv3 = tf.layers.conv2d(drop1, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    relu3 = tf.nn.relu(conv3)
    conv4 = tf.layers.conv2d(relu3, filters=100, kernel_size=(3, 3), 
                             padding='same', data_format='channels_first')
    pool2 = tf.layers.max_pooling2d(conv4, pool_size=(2, 2), strides=(2, 2), 
                                    padding='valid', data_format='channels_first')
    relu4 = tf.nn.relu(pool2)
    drop2 = tf.layers.dropout(relu4, 0.25, training=training)   
    
    flatten = tf.reshape(drop2, shape=[-1, 100*8*8])
    fc1 = tf.layers.dense(flatten, 512, activation=tf.nn.relu)
    drop3 = tf.layers.dropout(fc1, 0.5, training=training)
    logits = tf.layers.dense(drop3, N_CLASSES, name='output')
    return logits

In [5]:
def init_model(m):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate=LR, momentum=MOMENTUM)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 860 ms, sys: 527 ms, total: 1.39 s
Wall time: 1.38 s


In [7]:
%%time
# Place-holders
X = tf.placeholder(tf.float32, shape=[None, 3, 32, 32])
y = tf.placeholder(tf.int32, shape=[None])
training = tf.placeholder(tf.bool)  # Indicator for dropout layer
# Initialise model
sym = create_symbol(training)

CPU times: user 117 ms, sys: 463 µs, total: 118 ms
Wall time: 117 ms


In [8]:
%%time
model = init_model(sym)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# Accuracy logging
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 439 ms, sys: 373 ms, total: 812 ms
Wall time: 855 ms


In [9]:
%%time
# 173s
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label, training: True})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label, training: True})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.4375
1 Train accuracy: 0.5625
2 Train accuracy: 0.671875
3 Train accuracy: 0.75
4 Train accuracy: 0.75
5 Train accuracy: 0.6875
6 Train accuracy: 0.703125
7 Train accuracy: 0.859375
8 Train accuracy: 0.8125
9 Train accuracy: 0.734375
CPU times: user 2min 2s, sys: 14.1 s, total: 2min 16s
Wall time: 2min 53s


In [10]:
%%time
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym,1)
    output = sess.run(pred, feed_dict={X: data, training: False})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 3.63 s, sys: 842 ms, total: 4.47 s
Wall time: 4.41 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.776442307692
